## Home assignment 05: Bagging and OOB score

Please, fill the lines in the code below.
This is a simplified version of `BaggingRegressor` from `sklearn`. Please, notice, that `sklearn` API is **not preserved**.

Your algorithm should be able to train different instances of the same model class on bootstrapped datasets and to provide [OOB score](https://en.wikipedia.org/wiki/Out-of-bag_error) for the training set.

The model should be passed as model class with no explicit parameters and no parentheses.

Example:
```
import numpy as np
from sklearn.linear_model import LinearRegression

bagging_regressor = SimplifiedBaggingRegressor(num_bags=10, oob=True)
bagging_regressor.fit(LinearRegression, X, y)

```

In [1]:
import numpy as np

In [32]:
class SimplifiedBaggingRegressor:
    def __init__(self, num_bags, oob=False):
        self.num_bags = num_bags
        self.oob = oob #Out-of-bag error

    def _generate_splits(self, data: np.ndarray):
        '''
        Generate indices for every bag and store in self.indices_list list
        '''
        self.indices_list = []
        data_length = len(data)
        if self.oob:
            for bag in range(self.num_bags):
                indices = np.random.choice(data_length, size=data_length, replace=False)
                self.indices_list.append(indices)
        else:
            for bag in range(self.num_bags):
                indices = np.random.choice(data_length, size=data_length, replace=True)
                self.indices_list.append(indices)

    def fit(self, model_constructor, data, target):
        '''
        Fit model on every bag.
        Model constructor with no parameters (and with no ()) is passed to this function.

        example:

        bagging_regressor = SimplifiedBaggingRegressor(num_bags=10, oob=True)
        bagging_regressor.fit(LinearRegression, X, y)
        '''
        self.data = None
        self.target = None
        self._generate_splits(data)
        assert len(set(list(map(len, self.indices_list)))) == 1, 'All bags should be of the same length!'
        assert list(map(len, self.indices_list))[0] == len(data), 'All bags should contain `len(data)` number of elements!'

        self.models_list = []
        for bag in range(self.num_bags):
            model = model_constructor()
            data_bag, target_bag = data[self.indices_list[bag]], target[self.indices_list[bag]]  # Your Code Here
            self.models_list.append(model.fit(data_bag, target_bag)) # store fitted models here

        if self.oob:
            self.data = data
            self.target = target

    def predict(self, data):
        '''
        Get average prediction for every object from passed dataset
        '''
        # Your code here
        predictions = np.zeros(len(data))
        for model in self.models_list:
            predictions += model.predict(data)
        predictions /= len(self.models_list)
        return predictions

    def _get_oob_predictions_from_every_model(self):
        '''
        Generates list of lists, where list i contains predictions for self.data[i] object
        from all models, which have not seen this object during training phase
        '''
        list_of_predictions_lists = [[] for _ in range(len(self.data))]
        # Your Code Here

        for bag in range(self.num_bags):
            indices = self.indices_list[bag]
            mask = np.ones(len(self.data), dtype=bool)
            mask[indices] = False
            oob_data = self.data[mask]
            oob_target = self.target[mask]
            model = self.models_list[bag]
            if len(oob_data) > 0:
                predictions = model.predict(oob_data)
                print('*'*50)
                for i, index in enumerate(np.where(mask)[0]):
                    list_of_predictions_lists[index].append(predictions[i])

        #self.list_of_predictions_lists = np.array(list_of_predictions_lists, dtype=object)
        self.list_of_predictions_lists = list_of_predictions_lists

    def _get_averaged_oob_predictions(self):
        '''
        Compute average prediction for every object from training set.
        If object has been used in all bags on training phase, return None instead of prediction
        '''
        self._get_oob_predictions_from_every_model()
        n_models_used_for_oob = np.sum(self.list_of_predictions_lists != [])
        self.oob_predictions = np.zeros(len(self.data))
        for i in range(len(self.data)):
            if len(self.list_of_predictions_lists[i]) > 0:
                self.oob_predictions[i] = np.mean(self.list_of_predictions_lists[i])
            else:
                self.oob_predictions[i] = None

    def mean_squared_error(self, y_true, y_pred):
        '''
        Вычисляет среднеквадратичную ошибку между y_true и y_pred
        '''
        mse = ((y_true - y_pred) ** 2).mean()
        return mse

    def OOB_score(self):
        '''
        Compute mean square error for all objects, which have at least one prediction
        '''
        self._get_averaged_oob_predictions()
        # mask = ~np.isnan(self.oob_predictions)
        # mse = np.mean((self.target[mask] - self.oob_predictions[mask])**2)
        # return mse
        # for predictions in self.list_of_predictions_lists:
        #     print(predictions)
        oob_predictions = []
        for predictions in self.list_of_predictions_lists:
            if predictions: oob_predictions.append(np.mean(predictions))
            else: oob_predictions.append(0)
        oob_predictions = np.array(oob_predictions)
        oob_score = self.mean_squared_error(self.target, oob_predictions)
        return oob_score

In [95]:
import numpy as np

class SimplifiedBaggingRegressor:
    def __init__(self, num_bags, oob=False):
        self.num_bags = num_bags
        self.oob = oob

    def _generate_splits(self, data: np.ndarray):
        '''
        Generate indices for every bag and store in self.indices_list list
        '''
        self.indices_list = []
        data_length = len(data)
        # if self.oob:
        #     for bag in range(self.num_bags):
        #         indices = np.random.choice(data_length, size=data_length, replace=False)
        #         self.indices_list.append(indices)
        # else:
        for bag in range(self.num_bags):
            indices = np.random.choice(data_length, size=data_length, replace=True)
            self.indices_list.append(indices)

    def fit(self, model_constructor, data, target):
        '''
        Fit model on every bag.
        Model constructor with no parameters (and with no ()) is passed to this function.

        example:

        bagging_regressor = SimplifiedBaggingRegressor(num_bags=10, oob=True)
        bagging_regressor.fit(LinearRegression, X, y)
        '''
        self.data = None
        self.target = None
        self._generate_splits(data)
        assert len(set(list(map(len, self.indices_list)))) == 1, 'Все бэги должны быть одинаковой длины!'
        #assert list(map(len, self.indices_list))[0] == len(data), 'Каждый бэг должен содержать `len(data)` элементов!'
        self.models_list = []
        for bag in range(self.num_bags):
            model = model_constructor()
            indices = self.indices_list[bag]
            data_bag, target_bag = data[indices], target[indices]
            self.models_list.append(model.fit(data_bag, target_bag)) # сохраняем обученные модели здесь
        if self.oob:
            self.data = data
            self.target = target

    def predict(self, data):
        '''
        Получает среднее предсказание для каждого объекта из переданного набора данных
        '''
        predictions = np.zeros(len(data))
        for model in self.models_list:
            predictions += model.predict(data)
        predictions /= len(self.models_list)
        return predictions

    def _get_oob_predictions_from_every_model(self):
        '''
        Generates list of lists, where list i contains predictions for self.data[i] object
        from all models, which have not seen this object during training phase
        '''
        list_of_predictions_lists = [[] for _ in range(len(self.data))]
        # Your Code Here

        for bag in range(self.num_bags):
            indices = self.indices_list[bag]
            mask = np.ones(len(self.data), dtype=bool)
            mask[indices] = False
            oob_data = self.data[mask]
            model = self.models_list[bag]
            if len(oob_data) > 0:
                predictions = model.predict(oob_data)
                for i, index in enumerate(np.where(mask)[0]):
                    list_of_predictions_lists[index].append(predictions[i])

        self.list_of_predictions_lists = np.array(list_of_predictions_lists, dtype=object)

    def _get_averaged_oob_predictions(self):
        '''
        Вычисляет среднее предсказание для каждого объекта из тренировочного набора.
        Если объект использовался во всех бэгах во время обучения, возвращается None вместо предсказания.
        '''
        self._get_oob_predictions_from_every_model()
        n_models_used_for_oob = np.sum(self.list_of_predictions_lists != [])
        self.oob_predictions = np.zeros(len(self.data))
        for i in range(len(self.data)):
            if len(self.list_of_predictions_lists[i]) > 0:
                self.oob_predictions[i] = np.mean(self.list_of_predictions_lists[i])
            else:
                self.oob_predictions[i] = None

    def OOB_score(self):
        '''
        Вычисляет среднеквадратичную ошибку для всех объектов, у которых есть хотя бы одно предсказание
        '''
        self._get_averaged_oob_predictions()
        return np.nanmean((self.oob_predictions - self.target) ** 2)


### Local tests:

In [3]:
from sklearn.linear_model import LinearRegression
from tqdm.auto import tqdm

#### Simple tests:

In [96]:
for _ in tqdm(range(100)):
    X = np.random.randn(2000, 10)
    y = np.mean(X, axis=1)
    bagging_regressor = SimplifiedBaggingRegressor(num_bags=10, oob=True)
    bagging_regressor.fit(LinearRegression, X, y)
    predictions = bagging_regressor.predict(X)
    assert np.mean((predictions - y)**2) < 1e-6, 'Linear dependency should be fitted with almost zero error!'
    assert bagging_regressor.oob, 'OOB feature must be turned on'
    oob_score = bagging_regressor.OOB_score()
    assert oob_score < 1e-6, 'OOB error for linear dependency should be also close to zero!'
    assert abs(
        np.mean(
            list(map(len, bagging_regressor.list_of_predictions_lists))
        ) / bagging_regressor.num_bags - 1/np.exp(1)) < 0.1, 'Probability of missing a bag should be close to theoretical value!'

print('Simple tests done!')

  0%|          | 0/100 [00:00<?, ?it/s]

<ipython-input-95-b47f262ec07e>:85: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  n_models_used_for_oob = np.sum(self.list_of_predictions_lists != [])
<ipython-input-95-b47f262ec07e>:85: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  n_models_used_for_oob = np.sum(self.list_of_predictions_lists != [])
<ipython-input-95-b47f262ec07e>:85: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  n_models_used_for_oob = np.sum(self.list_of_predictions_lists != [])
<ipython-input-95-b47f262ec07e>:85: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  n_models_used_for_oob = np.sum(self.list_of_predictions_lists != [])
<ipython-input-95-b47f262ec07e>:85: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  n_models_used_for_oob = np.sum(self.list_of_predictions_lists != [])
<ipython-i

Simple tests done!


#### Medium tests

In [97]:
for _ in tqdm(range(10)):
    X = np.random.randn(200, 150)
    y = np.random.randn(len(X))
    bagging_regressor = SimplifiedBaggingRegressor(num_bags=20, oob=True)
    bagging_regressor.fit(LinearRegression, X, y)
    predictions = bagging_regressor.predict(X)
    average_train_error = np.mean((predictions - y)**2)
    assert bagging_regressor.oob, 'OOB feature must be turned on'
    oob_score = bagging_regressor.OOB_score()
    assert oob_score > average_train_error, 'OOB error must be higher than train error due to overfitting!'
    assert abs(
        np.mean(
            list(map(len, bagging_regressor.list_of_predictions_lists))
        ) / bagging_regressor.num_bags - 1/np.exp(1)) < 0.1, 'Probability of missing a bag should be close to theoretical value!'

print('Medium tests done!')

  0%|          | 0/10 [00:00<?, ?it/s]

<ipython-input-95-b47f262ec07e>:85: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  n_models_used_for_oob = np.sum(self.list_of_predictions_lists != [])
<ipython-input-95-b47f262ec07e>:85: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  n_models_used_for_oob = np.sum(self.list_of_predictions_lists != [])
<ipython-input-95-b47f262ec07e>:85: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  n_models_used_for_oob = np.sum(self.list_of_predictions_lists != [])
<ipython-input-95-b47f262ec07e>:85: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  n_models_used_for_oob = np.sum(self.list_of_predictions_lists != [])
<ipython-input-95-b47f262ec07e>:85: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  n_models_used_for_oob = np.sum(self.list_of_predictions_lists != [])
<ipython-i

Medium tests done!


<ipython-input-95-b47f262ec07e>:85: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  n_models_used_for_oob = np.sum(self.list_of_predictions_lists != [])


#### Complex tests:

In [98]:
for _ in tqdm(range(10)):
    X = np.random.randn(2000, 15)
    y = np.random.randn(len(X))
    bagging_regressor = SimplifiedBaggingRegressor(num_bags=100, oob=True)
    bagging_regressor.fit(LinearRegression, X, y)
    predictions = bagging_regressor.predict(X)
    oob_score = bagging_regressor.OOB_score()
    assert abs(
        np.mean(
            list(map(len, bagging_regressor.list_of_predictions_lists))
        ) / bagging_regressor.num_bags - 1/np.exp(1)) < 1e-2, 'Probability of missing a bag should be close to theoretical value!'

print('Complex tests done!')

  0%|          | 0/10 [00:00<?, ?it/s]

<ipython-input-95-b47f262ec07e>:85: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  n_models_used_for_oob = np.sum(self.list_of_predictions_lists != [])
<ipython-input-95-b47f262ec07e>:85: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  n_models_used_for_oob = np.sum(self.list_of_predictions_lists != [])
<ipython-input-95-b47f262ec07e>:85: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  n_models_used_for_oob = np.sum(self.list_of_predictions_lists != [])
<ipython-input-95-b47f262ec07e>:85: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  n_models_used_for_oob = np.sum(self.list_of_predictions_lists != [])
<ipython-input-95-b47f262ec07e>:85: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  n_models_used_for_oob = np.sum(self.list_of_predictions_lists != [])
<ipython-i

Complex tests done!


<ipython-input-95-b47f262ec07e>:85: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  n_models_used_for_oob = np.sum(self.list_of_predictions_lists != [])


In [99]:
np.mean(
            list(map(len, bagging_regressor.list_of_predictions_lists))
        ) / bagging_regressor.num_bags - 1/np.exp(1)

-0.0004944411714423169

Great job! Please, save `SimplifiedBaggingRegressor` to  `bagging.py` and submit your solution to the grading system!